In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
import itertools
from pathlib import Path
import os

2022-05-24 20:45:36.429429: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Tamanho imagem
img_rows, img_cols = 224, 224

input_shape = (img_rows, img_cols, 3)

# Batch size

batch_size = 32

# seed
seed = 42

path_dataset = Path('/scratch/dpfurtado/backup_home/corais/1_classificacao_train_50/dataset/1_train_50')
path_train = path_dataset/'train'
path_train_dev = path_dataset/'train-dev'

path_modelo = './model/resnet_plc_full_transfer/'

In [3]:
train_dev_dataset = image_dataset_from_directory(
        path_train_dev,
        image_size=(img_rows, img_cols),
        color_mode="rgb",
        batch_size=batch_size,
        shuffle=False)

Found 106 files belonging to 7 classes.


2022-05-24 20:45:42.800608: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-24 20:45:42.803800: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-24 20:45:44.595608: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-05-24 20:45:44.595775: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: pilheira
2022-05-24 20:45:44.595792: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: pilheira
2022-05-24 20:45:44.596228: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.129.6
2022-05-24 20:45:44.596313: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.103.1
2022-05-24 20:45:

In [4]:
AUTOTUNE = tf.data.AUTOTUNE
#Otimização
train_dev_dataset = train_dev_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = tf.keras.models.load_model(path_modelo)
y_pred_proba = model.predict(train_dev_dataset)
y_pred = np.argmax(y_pred_proba, axis=1)

2022-05-24 20:46:44.323460: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-24 20:46:44.347624: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2200000000 Hz


In [ ]:
y_train_dev = np.concatenate([y for x, y in train_dev_dataset], axis=0)

In [ ]:
from sklearn.metrics import log_loss
soft_preds=tf.nn.softmax(y_pred_proba).numpy()
log_loss(y_train_dev, soft_preds)

In [ ]:
np.save('./resultados/pred_proba_resnet101_plc.npy', soft_preds)

In [ ]:
acc = accuracy_score(y_train_dev, y_pred)
print("Acurácia: {:.4f}".format(acc))

In [ ]:
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
      cm (array, shape = [n, n]): a confusion matrix of integer classes
      class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Compute the labels from the normalized confusion matrix.
    labels = np.around(cm, decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

class_names = ['agaricia spp', 'favia gravida', 'millepora alcicornis', 'montastraea cavernosa', 'palythoa caribaeorum', 'porites astreoides', 'siderastrea stellata']

# Calculate the confusion matrix.
cm = confusion_matrix(y_train_dev, y_pred, normalize='true')
# Log the confusion matrix as an image summary.
figure = plot_confusion_matrix(cm,class_names)


In [ ]:
print(classification_report(y_train_dev, y_pred))

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef

In [ ]:
print(f1_score(y_train_dev, y_pred, average='macro'))
print(matthews_corrcoef(y_train_dev, y_pred))